<a href="https://colab.research.google.com/github/Arshad221b/Automatic-Music-Transcription/blob/master/CNN_predict_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!pip install pydrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

In [0]:
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()                         # Follow prompt in the authorization process
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()

In [0]:
drive = GoogleDrive(gauth)
your_module = drive.CreateFile({"id": "1oYnjFOJ0G3ipCvXVJ1x6XK9uuRrIonxx"})   # "your_module_file_id" is the part after "id=" in the shareable link
your_module.GetContentFile("utils.py")          # Save the .py module file to Colab VM
import utils  

In [89]:
import numpy as np
import keras
import cv2
from keras.layers import Dense, Flatten, Reshape, Input
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard, CSVLogger
from keras.models import load_model
from  skimage.measure import block_reduce
from PIL import Image
import utils
import pretty_midi as pm 
import os, os.path
import re

model_path = "/content/drive/My Drive/MainDATA/ckpt3.h5"
image_path = "/content/drive/My Drive/experiment/exp/conversion/"
#image_file = "song3_315.jpg"

midi_path = "/content/drive/My Drive/experiment/data2/song3_315.mid"
midi = pm.PrettyMIDI(midi_path)

one_hot = utils.pretty_midi_to_one_hot(midi)
np.savetxt("onehot_short1.csv",one_hot, delimiter=",")
y = utils.slice_to_categories(one_hot)

x = []

model = load_model(model_path)
model.compile(loss=keras.losses.binary_crossentropy,
            optimizer=keras.optimizers.Adam(),
            metrics=['accuracy'])
i = 0 
filenums = []
#for image_file in os.listdir(image_path):
#	s = str(image_file)
#	
#	s = str(image_file)
#	if s[:4] == '.jpg':
#		print(image_file)
#		print(s)
#		i  = i + 1
		
		#im = Image.open(os.path.join(image_path, image_file))
		#cv2.imshow(image_file)
#		im = im.crop((14, 13, 594, 301))
#		resize = im.resize((49, 145), Image.NEAREST)
#		resize.load()
	
#		arr = np.asarray(resize, dtype="float32")
	
#		x.append(arr)
#		filenums.append(int(re.search(r'\d+', image_file).group()))

#x = np.array(x)
#x /= 255.0
#print(x)


IsADirectoryError: ignored

In [86]:
import os
image_path = "/content/drive/My Drive/experiment/exp/conversion/"
for image_file in os.listdir(image_path):
	s = str(image_file)
	
	s = str(image_file)
	
	#	print(image_file)
	#	print(s)
	#	i  = i + 1
		
	im = Image.open(os.path.join(image_path, image_file))
	#cv2.imshow(image_file)
	im = im.crop((14, 13, 594, 301))
	resize = im.resize((49, 145))
	resize.load()
	
	arr = np.asarray(im, dtype="float32")
	
	x.append(arr)
	filenums.append(int(re.search(r'\d+', image_file).group()))
print(x)

IsADirectoryError: ignored

In [87]:
print(len(x))

365


In [88]:
x = np.array(x)
x /= 255.0

y_pred = model.predict(x)
print(np.nonzero(y))
print(y_pred)

notes_unsorted = [np.argmax(y_pred[n]) for n in range(len(y_pred))]

notes = [x for _,x in sorted(zip(filenums, notes_unsorted))]
print(notes)

i=0
for note in notes:
	one_hot = np.zeros((128, 25))
	one_hot[note, :] = 1
	mid = utils.one_hot_to_pretty_midi(one_hot)
	mid.write('daylight_' + str(i) + ".mid")
	i += 1

(array([69]),)
[[0.01191828 0.01126152 0.01035738 ... 0.01178274 0.00995758 0.01055533]
 [0.01304078 0.01127246 0.01162979 ... 0.0128296  0.0120298  0.01032847]
 [0.01310217 0.01352546 0.01264906 ... 0.01316804 0.01218084 0.01227352]
 ...
 [0.01510119 0.0123935  0.01277569 ... 0.01308978 0.01279747 0.01146933]
 [0.01316592 0.01086709 0.01120627 ... 0.01195741 0.01182568 0.01083598]
 [0.01295087 0.01215762 0.01227453 ... 0.01321182 0.01097491 0.01033545]]
[44, 49, 50, 50, 53, 55, 55, 55, 55, 55, 55, 55, 55, 57, 57, 57, 57, 57, 57, 57, 57, 57, 57, 57, 57, 57, 58, 58, 58, 58, 58, 58, 58, 58, 58, 58, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 61, 61, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 64, 64, 64, 64, 64, 64, 64, 64, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 

In [92]:
print(y)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]
